# FP Growth Algorithm

In [1]:
# we are going to import libraries to use in our program

import numpy as np
import operator as op
import time
import pandas as pd
import sklearn as skl
from itertools import compress,chain,combinations

In [2]:
# We will create a class names node which helps to store the FP-Tree

class Node():
    
    def __init__(self, item, support, parent = None, link =None):
        self.children = []
        self.link = link
        self.item = item
        self.support = support
        self.parent = parent
    
    def get_child_from_item(self, item):
        for node in self.children:
            if node.item == item:
                return node

        return None

    def add_child(self, alpha):
        self.children.append(alpha)

In [3]:
## Let us create our FP Tree class 
    
class FPTree():

    def __init__(self, minimum_support, minimum_confidence):
        self.min_sup = minimum_support
        self.min_conf = minimum_confidence
        self.frequent_itemsets = []
        self.root = None
        self.headers = {}
        self.frequent_frozensets = []
        self.dictionary_of_counts = {}
    
    def add_trns(self, transaction, header_table, node=None):
        if node == None:
            node = self.root
        for item in transaction[0]:
            node = self.add_node(item, node, transaction[1], header_table)
            
    def display_ourtre(self, node=None, indent_times=0):

        if not node:
            node = self.root
            
        print ("%s %s:%s" % ("    " * indent_times, node.item, node.support))
    
        for noder in node.children:
            child = noder
            self.display_ourtre(child, indent_times + 1) 
            

    def add_node(self, item ,node, count, hedr_tble):

        item_node = node.get_child_from_item(item)        

        if item_node:
            node = item_node
            node.support+=count
            return node
        else:
            node.add_child(Node(item,count,node))
            new_node = node.get_child_from_item(item)
            
            if hedr_tble[item][1]:
                node = hedr_tble[item][1]
                while node.link:
                    node = node.link
                node.link = new_node
            else:
                hedr_tble[item][1] = new_node
            return new_node
     

                
    # To find out the frequent itemsets in our code
        
    def get_frqt_itsts(self, trans, tail, trail=[], itemset={0}):
        
        freq_itemset = {}         
    # Creating frequent itemsets for conditional databases is a time-consuming task.
        itemset = {}
        
        for transaction in trans:
            for item in transaction[0]:
                itemset[item] = itemset.get(item,0) + transaction[1]

        for key in itemset:
            if itemset[key]>self.min_sup:
                freq_itemset[key] = itemset[key]
            if itemset[key]==self.min_sup:
                freq_itemset[key] = itemset[key]

    # Constructing a conditional FP Tree and obtaining the node links are the goals of this code below
        
        headers = self.constructFPTree(trans)
        list = []
        if tail:
            list.append(tail)
        if trail:
            for i in range(len(trail)):
                list.append(trail[i])

        if not len(list) <= 0:
            for i in freq_itemset:
                self.frequent_frozensets.append(frozenset([i] + list))
                self.dictionary_of_counts[frozenset([i] + list)] = freq_itemset[i]
        else:
            for i in freq_itemset:
                self.frequent_frozensets.append(frozenset([i]))
                self.dictionary_of_counts[frozenset([i])] = freq_itemset[i]  

    # To generate frequent itemsets, traverse the node links and recursively call the function to generate them.
        for head in sorted(headers.items(), key=lambda x: x[1][0]):

            trans = self.get_cond_db(head[1][1])
            self.get_frqt_itsts(trans, head[0] , list)

    # Function to construct FPTree
    
    # To traverse conditional fp tree and return transactions with frequent items
    
    def get_cond_db(self, head):
        lstr = []
        while not (head == None):      
            p = head
            trans = []
            count = p.support
            p = p.parent
            while not (not True):
                if p.item is None:
                    break
                trans.append(p.item)
                p = p.parent

    # to keep track of the number of transactions
            lstr = lstr + [[trans, count]]
            head = head.link
        return lstr
    
    def constructFPTree(self, trnsc):
        itemset = {}
        frequent_itemset = {}        
        
        for trns in trnsc:
            for item in trns[0]:
                itemset[item] = itemset.get(item,0) + trns[1]

        for hey in itemset:
            if itemset[hey]==self.min_sup:
                frequent_itemset[hey] = itemset[hey]
            if itemset[hey]>self.min_sup:
                frequent_itemset[hey] = itemset[hey]
           
        drTbl = {}
        for item in frequent_itemset:
            drTbl[item] = [frequent_itemset[item],None]
            
        if self.root == None:
            root = self.root = Node(None,0)       
        else:
            root = Node(None,0)

    # Traversing the database again and adding transactions to FPTree after sorting them acc. to item support
        for i in range(len(trnsc)):          
            trnsc[i][0] = [item for item in trnsc[i][0] if item in frequent_itemset]
            if not (len(trnsc[i][0])<=0):
                trnsc[i][0] = sorted(trnsc[i][0], key=lambda x: frequent_itemset[x], reverse = True)
                self.add_trns(trnsc[i], drTbl, root)
        
        if not len(self.frequent_itemsets) != 0:
            self.headers = drTbl
        return drTbl

In [4]:
# We have made the class of FPTree above!!! Hurray

# Let us now generate the association rules now


def association_rules(frozen_lstr, yo_dict, min_conf):
    
    print("\nThe Association rules are as follows: \n")
    
    No_of_association_rules=0
    
    for itr in frozen_lstr:
        k = checkout_subsets(itr)
        lstr = [frozenset(i) for i in k]
        del(lstr[0])
        lstr.remove(itr)
        countL = yo_dict[itr]

        for bby in lstr:
            ## checking the condition
            if(yo_dict[bby] >= countL * min_conf):
                set_comp = itr-bby
                print(list(bby), " ---> ", list(set_comp))
                No_of_association_rules+=1
                
    print("\nNo. of association rules are :",No_of_association_rules)
       
                

def checkout_subsets(ohh):
    return chain(*map(lambda x: combinations(ohh, x), range(0, len(ohh)+1)))
            

In [6]:
## Hurray we have made the classes for FP Growth tree.........
## Time to test our program on the input file

data_file = open("retail.txt")
trnstn = [[linee.split(' ')[:-1],1] for linee in data_file]

# provide the minimum support and minimum confidence values as input
    
minimum_support = float(input("Enter a minimum support count value : "))
minimum_confidence = float(input("Enter a minimum confidence value : "))

fpTree = FPTree(minimum_support, minimum_confidence)
Begin_time = time.time()

fpTree.get_frqt_itsts(trnstn, [])

End_time= time.time()

## printing the time needed 

print('Time taken by our FP Growth Algorithm is : ', End_time-Begin_time,'seconds\n') 

Number_of_frequent_items=0
for key in sorted(fpTree.dictionary_of_counts, key=lambda key: (len(key), fpTree.dictionary_of_counts[key])):
    Number_of_frequent_items+=1
    print (list(key))

print('\nNumber of frequent items above are :',Number_of_frequent_items)

association_rules(fpTree.frequent_frozensets, fpTree.dictionary_of_counts, fpTree.min_conf)

print("\nHooray!!! Succesfully implemented FP Tree Algorithm")

Enter a minimum support count value : 5000
Enter a minimum confidence value : 0.3
Time taken by our FP Growth Algorithm is :  0.473940372467041 seconds

['41']
['32']
['38']
['48']
['39']
['38', '48']
['48', '32']
['32', '39']
['41', '48']
['38', '39']
['41', '39']
['48', '39']
['48', '32', '39']
['38', '48', '39']
['41', '48', '39']

Number of frequent items above are : 15

The Association rules are as follows: 

['41']  --->  ['39']
['39']  --->  ['41']
['41']  --->  ['48']
['48']  --->  ['41']
['41']  --->  ['48', '39']
['48']  --->  ['41', '39']
['39']  --->  ['41', '48']
['41', '48']  --->  ['39']
['41', '39']  --->  ['48']
['48', '39']  --->  ['41']
['48']  --->  ['32']
['32']  --->  ['48']
['32']  --->  ['39']
['39']  --->  ['32']
['48']  --->  ['32', '39']
['32']  --->  ['48', '39']
['39']  --->  ['48', '32']
['48', '32']  --->  ['39']
['48', '39']  --->  ['32']
['32', '39']  --->  ['48']
['38']  --->  ['39']
['39']  --->  ['38']
['38']  --->  ['48']
['48']  --->  ['38']
['38']